In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.0.2


In [15]:
import pickle
import pandas as pd

In [16]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [17]:
def generate_uuids(df):
    ride_ids = []
    ride_ids = f'{year:04d}/{month:02d}_' + df.index.astype('str')
    return ride_ids

In [42]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [43]:
year = 2021
month = 2

fhv_file = f"https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year:04d}-{month:02d}.parquet"

In [46]:
df = read_data(fhv_file)

In [47]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [49]:
y_pred.mean()

16.191691679979066

# Question 2

In [50]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [51]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred

In [53]:
df_result.head()

,ride_id,predicted_duration
1,2021/02_1,14.539865
2,2021/02_2,13.740422
3,2021/02_3,15.593339
4,2021/02_4,15.188118
5,2021/02_5,13.817206


In [54]:
output_file = "homework4_df_results.parquet"

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)